In [42]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
    
df = pd.read_json("./openbeta-usa-routes-aug-2020.zip", lines=True)
# %matplotlib inline

In [43]:
df.sample(5)
# Get the number of rows and columns 
rows = len(df.axes[0]) 
cols = len(df.axes[1]) 
  
# Print the number of rows and columns 
print("Number of Rows: " + str(rows)) 
print("Number of Columns: " + str(cols)) 

df.head()

Number of Rows: 183253
Number of Columns: 9


,route_name,grade,safety,type,fa,description,location,protection,metadata
0,Wheres Waldo?,"{'YDS': 'V2', 'Font': '5+'}",,{'boulder': True},unknown,[Sit Start on the crack. Pull a big move to a ...,,[Pads],"{'left_right_seq': '999999', 'parent_lnglat': ..."
1,Unknown,{},,"{'tr': True, 'ice': True}",Unkown,[Just a general entry for the routes. Usually ...,[Can't miss the silo with a giant sheet of ice...,[No gear needed. All supplied and is Top Rope],"{'left_right_seq': '0', 'parent_lnglat': [-92...."
2,Vanished Edens,"{'YDS': 'V4', 'Font': '6B'}",,{'boulder': True},"Joe Feldman, 2019",[Start right hand in a sidepull slot and left ...,[Hot Stuff Camp Roof],[pad - good landing],"{'left_right_seq': '1', 'parent_lnglat': [-91...."
3,Stairway to Heaven,"{'YDS': '5.7', 'French': '5a', 'Ewbanks': '15'...",,"{'trad': True, 'tr': True}",unknown,[Climb the large flake right of Slot Machine t...,,"[SR, tricams are handy.]","{'left_right_seq': '5', 'parent_lnglat': [-91...."
4,Shagadelic Humper Bumper,"{'YDS': '5.8', 'French': '5b', 'Ewbanks': '16'...",,{'tr': True},unknown,[Climb the buttress left of Cake Walk.],,[Build a TR anchor on off of trees above.],"{'left_right_seq': '999999', 'parent_lnglat': ..."


In [44]:
# remove {}
df = df[df['grade'] != {}]

# remove all rows without YDS 

df = df[df['grade'].apply(lambda x: 'YDS' in x)]

def extract_yds(row):
    return row.get('YDS')


df['YDS'] = df['grade'].apply(extract_yds)

df.drop(columns=['grade'], inplace=True)

df.head()

,route_name,safety,type,fa,description,location,protection,metadata,YDS
0,Wheres Waldo?,,{'boulder': True},unknown,[Sit Start on the crack. Pull a big move to a ...,,[Pads],"{'left_right_seq': '999999', 'parent_lnglat': ...",V2
2,Vanished Edens,,{'boulder': True},"Joe Feldman, 2019",[Start right hand in a sidepull slot and left ...,[Hot Stuff Camp Roof],[pad - good landing],"{'left_right_seq': '1', 'parent_lnglat': [-91....",V4
3,Stairway to Heaven,,"{'trad': True, 'tr': True}",unknown,[Climb the large flake right of Slot Machine t...,,"[SR, tricams are handy.]","{'left_right_seq': '5', 'parent_lnglat': [-91....",5.7
4,Shagadelic Humper Bumper,,{'tr': True},unknown,[Climb the buttress left of Cake Walk.],,[Build a TR anchor on off of trees above.],"{'left_right_seq': '999999', 'parent_lnglat': ...",5.8
5,The Razor,,{'tr': True},unknown,"[On the North facing wall, start at the lowest...",,[Build TR anchor on trees above.],"{'left_right_seq': '999999', 'parent_lnglat': ...",5.8


In [45]:
all_keys = set().union(*(d.keys() for d in df['type']))

# Create separate columns for each key and fill them with boolean values
for key in all_keys:
    df[key] = df['type'].apply(lambda x: x.get(key, False))

# Clean the type column
df.drop(columns=['type'], inplace=True)

df.head()

,route_name,safety,fa,description,location,protection,metadata,YDS,alpine,trad,snow,mixed,ice,boulder,aid,sport,tr
0,Wheres Waldo?,,unknown,[Sit Start on the crack. Pull a big move to a ...,,[Pads],"{'left_right_seq': '999999', 'parent_lnglat': ...",V2,False,False,False,False,False,True,False,False,False
2,Vanished Edens,,"Joe Feldman, 2019",[Start right hand in a sidepull slot and left ...,[Hot Stuff Camp Roof],[pad - good landing],"{'left_right_seq': '1', 'parent_lnglat': [-91....",V4,False,False,False,False,False,True,False,False,False
3,Stairway to Heaven,,unknown,[Climb the large flake right of Slot Machine t...,,"[SR, tricams are handy.]","{'left_right_seq': '5', 'parent_lnglat': [-91....",5.7,False,True,False,False,False,False,False,False,True
4,Shagadelic Humper Bumper,,unknown,[Climb the buttress left of Cake Walk.],,[Build a TR anchor on off of trees above.],"{'left_right_seq': '999999', 'parent_lnglat': ...",5.8,False,False,False,False,False,False,False,False,True
5,The Razor,,unknown,"[On the North facing wall, start at the lowest...",,[Build TR anchor on trees above.],"{'left_right_seq': '999999', 'parent_lnglat': ...",5.8,False,False,False,False,False,False,False,False,True


In [46]:
# Clean Text
def extract_text(value):
    return value[0] if isinstance(value, list) and len(value) == 1 else None

df['description'] = df['description'].apply(extract_text)
df['location'] = df['location'].apply(extract_text)
df['protection'] = df['protection'].apply(extract_text)
df.drop(columns=['safety'], inplace=True)
df.head()

TypeError: 'method' object is not subscriptable

In [49]:
df = df.rename(columns={'fa': 'first_ascent'})
df.head()

,route_name,first_ascent,description,location,protection,metadata,YDS,alpine,trad,snow,mixed,ice,boulder,aid,sport,tr
0,Wheres Waldo?,unknown,Sit Start on the crack. Pull a big move to a c...,None,Pads,"{'left_right_seq': '999999', 'parent_lnglat': ...",V2,False,False,False,False,False,True,False,False,False
2,Vanished Edens,"Joe Feldman, 2019",Start right hand in a sidepull slot and left h...,Hot Stuff Camp Roof,pad - good landing,"{'left_right_seq': '1', 'parent_lnglat': [-91....",V4,False,False,False,False,False,True,False,False,False
3,Stairway to Heaven,unknown,Climb the large flake right of Slot Machine to...,None,"SR, tricams are handy.","{'left_right_seq': '5', 'parent_lnglat': [-91....",5.7,False,True,False,False,False,False,False,False,True
4,Shagadelic Humper Bumper,unknown,Climb the buttress left of Cake Walk.,None,Build a TR anchor on off of trees above.,"{'left_right_seq': '999999', 'parent_lnglat': ...",5.8,False,False,False,False,False,False,False,False,True
5,The Razor,unknown,"On the North facing wall, start at the lowest ...",None,Build TR anchor on trees above.,"{'left_right_seq': '999999', 'parent_lnglat': ...",5.8,False,False,False,False,False,False,False,False,True


In [56]:
all_keys = set().union(*(d.keys() for d in df['metadata']))

# Create separate columns for each key and fill them with boolean values
for key in all_keys:
    df[key] = df['metadata'].apply(lambda x: x.get(key, False))

# Clean the type column
df.drop(columns=['metadata'], inplace=True)

df.head()

,route_name,first_ascent,description,location,protection,YDS,alpine,trad,snow,mixed,ice,boulder,aid,sport,tr,mp_sector_id,mp_route_id,parent_sector,left_right_seq,parent_lnglat
0,Wheres Waldo?,unknown,Sit Start on the crack. Pull a big move to a c...,None,Pads,V2,False,False,False,False,False,True,False,False,False,118170033,118170758,Waldo's Rock Park,999999,"[-91.54207, 42.02717]"
2,Vanished Edens,"Joe Feldman, 2019",Start right hand in a sidepull slot and left h...,Hot Stuff Camp Roof,pad - good landing,V4,False,False,False,False,False,True,False,False,False,117944427,117944541,Hot Stuff Camp Roof,1,"[-91.5615, 42.6154]"
3,Stairway to Heaven,unknown,Climb the large flake right of Slot Machine to...,None,"SR, tricams are handy.",5.7,False,True,False,False,False,False,False,False,True,106947227,106956280,Drive In Wall,5,"[-91.5625, 42.614]"
4,Shagadelic Humper Bumper,unknown,Climb the buttress left of Cake Walk.,None,Build a TR anchor on off of trees above.,5.8,False,False,False,False,False,False,False,False,True,106947239,106956324,Cake Walk and Razor Wall,999999,"[-91.5625, 42.614]"
5,The Razor,unknown,"On the North facing wall, start at the lowest ...",None,Build TR anchor on trees above.,5.8,False,False,False,False,False,False,False,False,True,106947239,106956330,Cake Walk and Razor Wall,999999,"[-91.5625, 42.614]"


In [58]:
# column_headings = df.columns.tolist()
# print(column_headings)

# Function to extract longitude and latitude
def extract_longitude(row):
    return row[0] if isinstance(row, list) and len(row) == 2 else None

def extract_latitude(row):
    return row[1] if isinstance(row, list) and len(row) == 2 else None

# Create new columns 'longitude' and 'latitude' using the extracted values
df['longitude'] = df['parent_lnglat'].apply(extract_longitude)
df['latitude'] = df['parent_lnglat'].apply(extract_latitude)

# Drop the original 'parent_lnglat' column
df.drop(columns=['parent_lnglat'], inplace=True)

df.head()

,route_name,first_ascent,description,location,protection,YDS,alpine,trad,snow,mixed,...,boulder,aid,sport,tr,mp_sector_id,mp_route_id,parent_sector,left_right_seq,longitude,latitude
0,Wheres Waldo?,unknown,Sit Start on the crack. Pull a big move to a c...,None,Pads,V2,False,False,False,False,...,True,False,False,False,118170033,118170758,Waldo's Rock Park,999999,-91.54207,42.02717
2,Vanished Edens,"Joe Feldman, 2019",Start right hand in a sidepull slot and left h...,Hot Stuff Camp Roof,pad - good landing,V4,False,False,False,False,...,True,False,False,False,117944427,117944541,Hot Stuff Camp Roof,1,-91.56150,42.61540
3,Stairway to Heaven,unknown,Climb the large flake right of Slot Machine to...,None,"SR, tricams are handy.",5.7,False,True,False,False,...,False,False,False,True,106947227,106956280,Drive In Wall,5,-91.56250,42.61400
4,Shagadelic Humper Bumper,unknown,Climb the buttress left of Cake Walk.,None,Build a TR anchor on off of trees above.,5.8,False,False,False,False,...,False,False,False,True,106947239,106956324,Cake Walk and Razor Wall,999999,-91.56250,42.61400
5,The Razor,unknown,"On the North facing wall, start at the lowest ...",None,Build TR anchor on trees above.,5.8,False,False,False,False,...,False,False,False,True,106947239,106956330,Cake Walk and Razor Wall,999999,-91.56250,42.61400


In [59]:
df.head()

,route_name,first_ascent,description,location,protection,YDS,alpine,trad,snow,mixed,...,boulder,aid,sport,tr,mp_sector_id,mp_route_id,parent_sector,left_right_seq,longitude,latitude
0,Wheres Waldo?,unknown,Sit Start on the crack. Pull a big move to a c...,None,Pads,V2,False,False,False,False,...,True,False,False,False,118170033,118170758,Waldo's Rock Park,999999,-91.54207,42.02717
2,Vanished Edens,"Joe Feldman, 2019",Start right hand in a sidepull slot and left h...,Hot Stuff Camp Roof,pad - good landing,V4,False,False,False,False,...,True,False,False,False,117944427,117944541,Hot Stuff Camp Roof,1,-91.56150,42.61540
3,Stairway to Heaven,unknown,Climb the large flake right of Slot Machine to...,None,"SR, tricams are handy.",5.7,False,True,False,False,...,False,False,False,True,106947227,106956280,Drive In Wall,5,-91.56250,42.61400
4,Shagadelic Humper Bumper,unknown,Climb the buttress left of Cake Walk.,None,Build a TR anchor on off of trees above.,5.8,False,False,False,False,...,False,False,False,True,106947239,106956324,Cake Walk and Razor Wall,999999,-91.56250,42.61400
5,The Razor,unknown,"On the North facing wall, start at the lowest ...",None,Build TR anchor on trees above.,5.8,False,False,False,False,...,False,False,False,True,106947239,106956330,Cake Walk and Razor Wall,999999,-91.56250,42.61400


In [ ]:
# Run this only once

from sqlalchemy import create_engine 
import os

uri = os.environ['DB_URL']
# print(uri)
if uri.startswith("postgres://"):
    uri = uri.replace("postgres://", "postgresql://", 1)
# print('After: ' + uri)
engine = create_engine(uri)
df.to_sql('table_name', engine)


In [63]:
query = f"SELECT * FROM table_name LIMIT 5"
df = pd.read_sql(query, engine)

# Display the first 5 rows of the retrieved data
print(df)

   index                route_name       first_ascent  \
0      0             Wheres Waldo?            unknown   
1      2            Vanished Edens  Joe Feldman, 2019   
2      3        Stairway to Heaven            unknown   
3      4  Shagadelic Humper Bumper            unknown   
4      5                 The Razor            unknown   

                                         description             location  \
0  Sit Start on the crack. Pull a big move to a c...                 None   
1  Start right hand in a sidepull slot and left h...  Hot Stuff Camp Roof   
2  Climb the large flake right of Slot Machine to...                 None   
3              Climb the buttress left of Cake Walk.                 None   
4  On the North facing wall, start at the lowest ...                 None   

                                 protection  YDS  alpine   trad   snow  ...  \
0                                      Pads   V2   False  False  False  ...   
1                        pad - good 